 ## Create the data

In [1]:
# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis.enums import TimestampFormat
from mimesis import Datetime

dt = Datetime()
f = Field(locale=Locale.EN_GB, seed=42)


# Add table definitions

table_definition = {
    "person":{
        "amount":1000
    },
    "product":{
        "amount":1000
    },
    "order":{
        "amount":10000
    },
    "artist":{
        "amount":500
    },
    "review":{
        "amount":2000
    },
    "create":{
        "amount":1000 # should be same as product
    }
}

# Create table data

## person

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "id":"person:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person']['amount']
)
person_data = person_schema.create()

person_id_count = table_definition['person']['amount']-1

print("person data created")

## product

def product_generator() -> dict:
    return {
        "id":"product:"+f"⟨{f('uuid')}⟩",
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=f('integer_number', start=8, end=25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":f('integer_number', start=0, end=20), 
        "image_url":f('stock_image_url')
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product']['amount']
)

product_data = product_schema.create()

product_id_count = table_definition['product']['amount']-1

print("product data created")

## order

def order_generator() -> dict:
    person_number = f('integer_number', start=0, end=person_id_count)
    product_number = f('integer_number', start=0, end=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    return {
        "id":f"order:"+f"⟨{f('uuid')}⟩",
        "in":person_data[person_number]['id'],
        "out":product_data[product_number]['id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity":f('integer_number', start=1, end=3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order']['amount']
)

order_data = order_schema.create()

order_id_count = table_definition['order']['amount']-1

print("order data created")

## artist

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "id":"artist:"+f"⟨{f('uuid')}⟩",
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }


artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist']['amount']
)

artist_data = artist_schema.create()

artist_id_count = table_definition['artist']['amount']-1

print("artist data created")

## review

def review_generator() -> dict:
    return {
        "id":"review:"+f"⟨{f('uuid')}⟩",
        "person":person_data[f('integer_number', start=0, end=person_id_count)]['id'],
        "product":product_data[f('integer_number', start=0, end=product_id_count)]['id'],
        "artist":artist_data[f('integer_number', start=0, end=artist_id_count)]['id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=f('integer_number', start=8, end=50)))
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review']['amount']
)

review_data = review_schema.create()

review_id_count = table_definition['review']['amount']-1

print("review data created")

## create
def create_generator() -> dict:
    product_number = f('increment')-1
    artist_number = f('integer_number', start=0, end=artist_id_count)
    created_at = dt.timestamp(TimestampFormat.ISO_8601, start=2023, end=2023)
    return {
        "id":"create:"+f"⟨{f('uuid')}⟩",
        "in":artist_data[artist_number]['id'],
        "out":product_data[product_number]['id'],
        "created_at":created_at,
        "quantity":product_data[product_number]['quantity']
    }

create_schema = Schema(
    schema=create_generator,
    iterations=table_definition['create']['amount']
)

create_data = create_schema.create()

print("create data created")

person data created
product data created
order data created
artist data created
review data created
create data created


 ## Load the data

In [3]:
from surrealdb import SurrealDB
from uuid import uuid4

db = SurrealDB("ws://localhost:8000/test/test")

db.signin({
    "username": "root",
    "password": "root",
})


def insert_relate_statement(table_data:list[dict]) -> str:
    """
    Inserting data through relate statement
    """

    table_record_id = -1
    for record in table_data:
        table_record_id += 1
        db.query(
    f"RELATE {table_data[table_record_id]['in']} -> {table_data[table_record_id]['id']} -> {table_data[table_record_id]['out']} CONTENT {record};"
            )

In [ ]:
%%time
db.query(f"INSERT INTO person {person_data}")

In [ ]:
%%time
db.query(f"INSERT INTO product {product_data}")

In [ ]:
%%time
insert_relate_statement(order_data)

In [ ]:
%%time
db.query(f"INSERT INTO artist {artist_data}")

In [ ]:
%%time
db.query(f"INSERT INTO review {review_data}")

In [4]:
%%time
db.query(f"INSERT INTO create {create_data}")

CPU times: user 350 ms, sys: 11 ms, total: 361 ms
Wall time: 413 ms


[{'created_at': '2023-02-01T17:11:38.974415',
  'id': 'create:⟨1d0cbfb3-40b3-485a-96bd-2b4de6e51a3b⟩',
  'in': 'artist:⟨49958a36-2dce-4b45-9b67-ad35ac314709⟩',
  'out': 'product:⟨d6b20019-2a87-4b62-a878-d69d15a81d1a⟩',
  'quantity': 8},
 {'created_at': '2023-09-16T03:24:45.103881',
  'id': 'create:⟨3d3e2dcc-b832-4e5e-8dd5-8be476cbe6bd⟩',
  'in': 'artist:⟨3ce3a733-42ca-4e39-8827-eae45cd82e79⟩',
  'out': 'product:⟨73ba87fd-3a8c-4f1c-bdb7-5ead7fa718c4⟩',
  'quantity': 3},
 {'created_at': '2023-03-23T04:36:35.611386',
  'id': 'create:⟨75c741c1-454d-4854-a75e-cd4f7e5337ef⟩',
  'in': 'artist:⟨10627037-0d8a-49de-9bf5-f61976a8c24a⟩',
  'out': 'product:⟨844860c4-a73c-4883-8b49-39d24471e76d⟩',
  'quantity': 13},
 {'created_at': '2023-12-19T18:14:36.801639',
  'id': 'create:⟨c266f200-a0ec-4a16-a3b5-4176d5a0e80c⟩',
  'in': 'artist:⟨a02cb703-587d-46a3-bc17-eb907e7b2126⟩',
  'out': 'product:⟨39f7ca8f-9722-440b-950a-1fd3a540d93b⟩',
  'quantity': 6},
 {'created_at': '2023-06-20T06:26:56.622409',
  'id

 ## Run the queries

In [ ]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = db.query(f"SELECT VALUE id FROM person")
product_ids = db.query(f"SELECT VALUE id FROM product")
order_ids = db.query(f"SELECT VALUE id FROM order")
artist_ids = db.query(f"SELECT VALUE id FROM artist")
review_ids = db.query(f"SELECT VALUE id FROM review")

 ### Q1: lookup vs record links

In [ ]:
%%timeit
list(db.query(""" 
SELECT
    id,
    rating,
    review_text,
	artist.name,
	artist.email,
	artist.phone,
	person.name,
	person.email,
	person.phone,
	product.name,
	product.category,
	product.price
FROM review;
"""))

 ### Q2: lookup vs graph

In [ ]:
%%timeit
db.query(""" 
SELECT
	price,
	order_date,
	product_name,
	->product.category,
	->product.description,
	->product.image_url,
	<-person.name,
	<-person.email,
	<-person.phone,
	->product<-create<-artist.name,
	->product<-create<-artist.email,
	->product<-create<-artist.phone
FROM order;
""")

 ### Q2: lookup vs graph - using in/out instead of arrow


In [ ]:
%%timeit
db.query(""" 
SELECT
	price,
	order_date,
	product_name,
	out.category,
	out.description,
	out.image_url,
	in.person.name,
	in.person.email,
	in.person.phone,
	out<-create.in.name,
	out<-create.in.email,
	out<-create.in.phone
FROM order;
""")

 ### Q3: Name and email for all customers in England

In [ ]:
%%time
db.query(""" 
DEFINE INDEX person_country ON TABLE person COLUMNS address.country;
""")

In [ ]:
%%timeit
db.query(""" 
SELECT name, email 
FROM person 
WHERE address.country = "England";	
""")

 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [ ]:
%%timeit
db.query(""" 
SELECT count() FROM order
WHERE (order_status != "pending"
OR order_status = null)
AND time::month(<datetime>order_date) <=3
AND ->product<-create<-artist.address.country ?= "England"
GROUP ALL;
""")


 ### Q5: Delete a specific review

In [ ]:
%%time
db.query(f""" 
DELETE {review_ids[0]};
""")


 ### Q6: Delete reviews from a particular category

In [ ]:
%%time
db.query(""" 
DELETE review
WHERE product.category = "charcoal";
""")

 ### Q7: Update a customer address

In [ ]:
%%timeit
db.query(f"UPDATE {person_ids[randint(0, person_id_count)]}"+"""
SET address = {
	'address_line_1': '497 Ballycander',
	'address_line_2': None,
	'city': 'Bromyard',
	'country': 'Wales',
	'post_code': 'ZX8N 4VJ',
	'coordinates': [68.772592, -35.491877]
	}
RETURN NONE;
""")

 ### Q8: Update discounts for products

In [ ]:
%%time
db.query(""" 
UPDATE product
SET discount = 0.2
WHERE price < 1000
RETURN NONE;
""")


 ### Q9: Transaction - order from a new customer

In [ ]:
%%timeit
random_person_id = person_ids[randint(0, person_id_count)]
random_product_id = product_ids[randint(0, product_id_count)]

db.query(""" 
# Transaction - order from a new customer
BEGIN TRANSACTION;
-- insert into the person table
CREATE person CONTENT {
	"""+f"'id': {random_person_id},"+"""
	'first_name': 'Karyl',
	'last_name': 'Langley',
	'name': 'Karyl Langley',
	'company_name': None,
	'email': 'dee1961@gmail.com',
	'phone': '+44 47 3516 5895',
	'address': {
		'address_line_1': '510 Henalta',
		'address_line_2': None,
		'city': 'Lyme Regis',
		'country': 'Northern Ireland',
		'post_code': 'TO6Q 8CM',
		'coordinates': [-34.345071, 118.564172]
		}
	};

-- relate into the order table"""+
f"RELATE {random_person_id} -> order:uuid() -> {random_product_id}"+"""
CONTENT {
        "currency": "£",
        "discount": ->product.discount,
        "order_date": time::now(),
        "order_status": "pending",
        "payment_method": "PayPal",
        "price": ->product.price,
        "product_name": ->product.name,
        "quantity": 1,
        "shipping_address": <-person.address
	};

-- update the product table to reduce the quantity"""+
f"""
UPDATE {random_product_id} SET quantity -= 1 RETURN NONE;
COMMIT TRANSACTION;
""")


 ### Q10: "Transaction"* - New Artist creates their first product

In [ ]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = str(uuid4())
new_product_id = str(uuid4())

db.query(""" 
BEGIN TRANSACTION;
-- insert into the artist table
CREATE artist CONTENT {"""+
        f"'id': 'artist:⟨{new_artist_id}⟩',"+"""
        'first_name': 'Anderson',
        'last_name': 'West',
        'name': 'Anderson West',
        'company_name': 'Atkins(ws) (ATK)',
        'email': 'six1933@gmail.com',
        'phone': '056 5881 1126',
        'address': {
                'address_line_1': '639 Connaugh',
                'address_line_2': None,
                'city': 'Ripon',
                'country': 'Scotland',
                'post_code': 'CG3U 4TH',
                'coordinates': [4.273648, -112.907273]
                }
        };

-- insert into the product table
CREATE product CONTENT {"""+
        f"'id': 'product:⟨{new_product_id}⟩',"+"""
        'name': 'managed edt allocated pda',
        'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
        'category': 'watercolor',
        'price': 15735.96,
        'currency': '£',
        'discount': None,
        'quantity': 1,
        'image_url': 'https://source.unsplash.com/1920x1080?'
        };

-- relate into the create table"""+
f"RELATE artist:⟨{new_artist_id}⟩ -> create:uuid() -> product:⟨{new_product_id}⟩"+"""
CONTENT {
        created_at: time::now(),
        quantity: 1
};
COMMIT TRANSACTION;
""")